In [ ]:
# Importing essential libraries
from flask import Flask, render_template, request
import pickle
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import os

# Load the DNN model
classifier = tf.keras.models.load_model('DNN_Model.model')
#Prepare Data
def prepare_data(data):
    scaler = MinMaxScaler(copy=True, feature_range=(0, 1))
    train_df_Scal = scaler.fit_transform(data)
    X = pd.DataFrame(train_df_Scal)
    X_New = X.to_numpy()
    X_New= X_New.reshape(X_New.shape[0], X_New.shape[1], 1)
    return X_New
def Preapare_DataSet(fileName):
    data_frame = pd.read_csv(fileName)
    columns_d=[]
    for col in data_frame.columns:
        columns_d.append(col)
    data = data_frame.values.tolist()
    return columns_d , data
app = Flask(__name__)

@app.route('/')
def home():
	return render_template('index.html')

@app.route('/predict', methods=['POST'])
#Predictions
def predict():
    if request.method == 'POST':
        wc = float(request.form['WC'])
        duw = float(request.form['DUW'])
        gc = float(request.form['GC'])
        af = float(request.form['AF'])
        c = float(request.form['C'])
        lb = float(request.form['LB'])
        lkpa = float(request.form['LKpa'])
        tmin = float(request.form['TMin'])
        db = float(request.form['DB'])
        
        data1 = pd.DataFrame({
            'WC'  :[14.00    , 9.28    , 3.20  , wc   ],
            'DUW' :[17.80    , 16.09   , 13.50 , duw  ],
             'GC' :[55.00    , 15.40   , 0.00  , gc   ],
            'AF'  :[49.10    , 41.75   , 31.00 , af   ],
            'C'   :[11.00    , 2.15    , 0.00  , c    ],
            'LB'  :[6.40     , 4.20    , 1.00  , lb   ],
            'LKpa':[703.00   , 171.64  , 1.92  , lkpa ],
            'TMin':[10949.60 , 2547.90 , 7.00  , tmin ],
            'DB'  :[2.00     , 0.93    , 0.00  , db   ]

        })
        data=prepare_data(data1)
       
        
        my_prediction = classifier.predict(data)
        my_prediction0 = round(my_prediction[3][0],3)
        return render_template('result.html', prediction_text=my_prediction0 )

@app.route('/datasetDNN', methods=['GET', 'POST'])
def datasetDNN():
    columns , data =Preapare_DataSet("DataPredict/DNN_Model.csv")
    return render_template("dataset_table.html",headings=columns,data=data )
@app.route('/datasetLR', methods=['GET', 'POST'])
def datasetLR():
    columns , data =Preapare_DataSet("DataPredict/LR_Model.csv")
    return render_template("dataset_table.html",headings=columns,data=data )
@app.route('/datasetEns', methods=['GET', 'POST'])
def datasetEns():
    columns , data =Preapare_DataSet("DataPredict/Ensembel_model.csv")
    return render_template("dataset_table.html",headings=columns,data=data )
@app.route('/datasetEns', methods=['GET', 'POST'])
def datasetLasso():
    columns , data =Preapare_DataSet("DataPredict/Lasso_model.csv")
    return render_template("dataset_table.html",headings=columns,data=data )
@app.route('/datasetRidge', methods=['GET', 'POST'])
def datasetRidge():
    columns , data =Preapare_DataSet("DataPredict/Ridge_model.csv")
    return render_template("dataset_table.html",headings=columns,data=data )
@app.route('/datasetSVR', methods=['GET', 'POST'])
def datasetSVR():
    columns , data =Preapare_DataSet("DataPredict/SVR_model.csv")
    return render_template("dataset_table.html",headings=columns,data=data )
@app.route('/datasetANN', methods=['GET', 'POST'])
def datasetANN():
    columns , data =Preapare_DataSet("DataPredict/MLPRegressorModel_model.csv")
    return render_template("dataset_table.html",headings=columns,data=data )
if __name__ == '__main__':
    from waitress import serve
    serve(app, host="0.0.0.0", port=8080)